In [10]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, when

In [11]:
spark = SparkSession.builder \
    .appName("BronzeToSilver") \
    .getOrCreate()

In [12]:
bronze_path = "../../datalake/bronze"
silver_path = "../../datalake/silver"

In [14]:
class datacleaner:
    @staticmethod
    def limpeza_basica(df):
        df = df.dropDuplicates()
        df = df.na.drop(how='all')
        for col_name in df.columns:
            if "data" in col_name or "mes" in col_name:
                df = df.withColumn(col_name,
                    when(col(col_name).rlike(r'^\d{4}-\d{2}-\d{2}$'), to_date(col(col_name), 'yyyy-MM-dd'))
                    .otherwise(None))
        return df


In [15]:
for nome_arquivo in os.listdir(bronze_path):
    caminho = os.path.join(bronze_path, nome_arquivo)
    df = spark.read.parquet(caminho)

    df = datacleaner.limpeza_basica(df)

    destino_silver = os.path.join(silver_path, nome_arquivo)
    df.write.mode("overwrite").parquet(destino_silver)


25/06/17 18:24:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/06/17 18:24:55 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/06/17 18:24:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
